# ILIAD CELL TOWERS GRAPH

In [1]:
# Importing all libraries

import reverse_geocode as rgeo  # used to find the locations of the cells
from neo4j import GraphDatabase as graph  # used to connect to the neo4j database
import pandas as pd
import datetime as dt 
import random as rd
import os
from dotenv import dotenv_values
from db_data import DatabaseData

bolt://localhost:7687


# CONNECTING TO THE NEO4J DATABASE

In [2]:
# Connecting to the Neo4j Database 

db = DatabaseData()

# CHANGE THIS VALUES TO CONNECT TO YOUR OWN DATABASE/ACCOUNT

DB_USER = db.user # name of the database
DB_LINK = db.connection  # link to the database
DB_PASSWORD = db.password # password of the database

In [3]:
# Getting the csv with all the informations
df = pd.read_csv('iliad.csv', sep=';')

/tmp/ipykernel_32418/462349057.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('iliad.csv', sep=';')


In [4]:
if __name__ == '__main__':
    driver = graph.driver(
        DB_LINK,
        auth=(DB_USER, DB_PASSWORD)
    )

In [5]:
# Function user to creare people and users of
def crea_person_user(name):
    driver.execute_query(
        f'INSERT (:Person :User{{name: $personal_name}})',
        {"personal_name": name}
    )

In [6]:
# Create a cell to connect a device to
def create_cell(name, code):
    driver.execute_query(
        f'MERGE (:Cell {{name: $personal_name, cell_code: $code}} )',
        {"personal_name": name, "code": code}
    )

In [ ]:
# This cycle creates N nodes (:Cell {codice}) for a site
for node, cid, cell in zip(df['node_id'], df['cid'], df['cell_name']):
    codice = f'{node}:{cid}'
    create_cell(cell, codice)

In [ ]:
# Create a site where there will be cells for devices to connect to
def crea_sito(nome, latitude, longitude, codice_id):
    driver.execute_query(
        f'MERGE (:Sito {{name: $nome, lat: $lat, long: $long, codice_identificativo: $codice_id}} )',
        {"nome": nome, "lat": latitude, "long": longitude, "codice_id": codice_id}
    )

In [ ]:
# Create a relation [:APPARTIENE_A] between the site and the cell 
def collega_sito_cella(id):
    id_stringa = str(id)
    driver.execute_query(
        f"MATCH (x:Cella) WHERE x.cell_code STARTS WITH $id_s MATCH (y:Sito) WHERE y.codice_identificativo = $id MERGE (x)-[:APPARTIENE_A]->(y)",
        {"id_s": id_stringa, "id": id})

In [ ]:
# Create geolocation 
def crea_geoloc(loc_diz):
    city = loc_diz['city']
    regione = loc_diz['state']
    provincia = loc_diz['county']
    driver.execute_query(
        f"MERGE (x: Città {{nome_città: $nome_c}}) MERGE (y: Provincia {{nome_provincia: $nome_p}}) MERGE (z: Regione {{nome_regione: $nome_n}}) MERGE (x)-[:IN]->(y)  MERGE (y)-[:IN]->(z)" ,
        {"nome_c": city, "nome_p": provincia,"nome_n": regione}
    )

In [ ]:
def collega_loc_sito(vet):
    driver.execute_query(
        f"MATCH (s:Sito) WHERE s.codice_identificativo = $codice MATCH (c:Città) WHERE c.nome_città = $nome MERGE (s)-[:IN]->(c)",
        {"codice": vet[0],"nome": vet[1]}
    )

In [ ]:
def crea_sim():
    sim_code = rd.randint(10**13,10**14-1)
    return sim_code

In [ ]:
def generate_call():
    now = dt.datetime.now()
    start_date = now - dt.timedelta(days=rd.randint(0, 7))
    start_time = dt.time(hour=rd.randint(0, 23), minute=rd.randint(0, 59))
    while True:
        end_time = dt.time(hour=rd.randint(0, 23), minute=rd.randint(0, 59))
        if end_time > start_time:
            break
    call_start_time = dt.datetime.combine(start_date, start_time)
    call_end_time = dt.datetime.combine(start_date, end_time)

    return call_start_time, call_end_time

In [ ]:
def select_random_cell(colonna_df):
    random_cella = np.random.choice(colonna_df)
    return random_cella

In [ ]:
def create_call_nodes():
    sim = crea_sim()
    start_time, end_time = generate_call()
    nome_cella_random = select_random_cell(ddf['cell_name'])
    lista_ping, metadata, keys = driver.execute_query(
      """
      MATCH (c :Cella {name: $nome }) 
      MERGE (s :Sim {codice_sim: $sim_id}) 
      WITH c,s 
      MATCH q = (s)-[p:PING WHERE (p.inizio_chiamata >= $inizio_ping AND p.inizio_chiamata <= $fine_ping) OR (p.fine_chiamata >= $inizio_ping AND p.fine_chiamata <= $fine_ping) ]->(c)  
      RETURN q
      """,
      {'sim_id': sim, 'inizio_ping': start_time, 'fine_ping': end_time, 'nome': nome_cella_random}
    )
    if not lista_ping:
        driver.execute_query(
            """
            MATCH (c :Cella {name: $nome }) 
            MATCH (s :Sim {codice_sim: $sim_id})
            CREATE q = (s)-[p:PING {inizio_chiamata: $inizio_ping, fine_chiamata: $fine_ping}]->(c) 
            """,
            {'sim_id': sim, 'inizio_ping': start_time, 'fine_ping': end_time, 'nome': nome_cella_random}
        )